In [1]:
# import libs

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# book website url
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

In [3]:
# https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [4]:
# determine rating of a book
def determine_rating(tags):
    if "star-rating One" in str(tags):
        return 1
    elif "star-rating Two" in str(tags):
        return 2
    elif "star-rating Three" in str(tags):
        return 3
    elif "star-rating Four" in str(tags):
        return 4
    elif "star-rating Five" in str(tags):
        return 5
    else:
        return 0

In [5]:
df = pd.DataFrame(columns=['Title', 'Price', 'Rating','Stock'])

In [6]:
# scrape website and get the information
for index in range(1,51):
    page = requests.get(base_url.format(index), headers=headers)
    soup = BeautifulSoup(page.text, "lxml")
    n = len(soup.select(".product_pod"))
    for i in range(n):
        title = soup.select(".product_pod")[i].find('h3').find('a')['title']
        price = float(soup.select(".product_pod")[i].find(class_="price_color").get_text()[2:-1])
        stock = soup.select(".product_pod")[i].find(class_="instock availability").get_text().replace("\n","").strip()
        rating = determine_rating(soup.select(".product_pod")[i])
        df = df.append({'title':title, 'price':price, 'rating':rating,'stock':stock}, ignore_index=True)    

In [7]:
df = df[['title', 'price', 'rating', 'stock']]

In [8]:
df.head()

,title,price,rating,stock
0,A Light in the Attic,51.7,3.0,In stock
1,Tipping the Velvet,53.7,1.0,In stock
2,Soumission,50.1,1.0,In stock
3,Sharp Objects,47.8,4.0,In stock
4,Sapiens: A Brief History of Humankind,54.2,5.0,In stock


In [9]:
# save to file
df.to_csv("BookWebScraperDataset.csv",index=False)